In [1]:
pip install mysql-connector-python

In [ ]:
import tkinter as tk
from tkinter import messagebox
import mysql.connector
from datetime import datetime

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",  # <- Replace this
    database="LibraryDB"
)
cursor = conn.cursor()

# Functions
def add_book():
    name = entry_book_name.get()
    author = entry_author.get()
    if not name or not author:
        messagebox.showerror("Error", "All fields required!")
        return
    cursor.execute("INSERT INTO Books (book_name, author) VALUES (%s, %s)", (name, author))
    conn.commit()
    messagebox.showinfo("Success", "Book added!")

def add_member():
    name = entry_member.get()
    contact = entry_contact.get()
    if not name or not contact:
        messagebox.showerror("Error", "All fields required!")
        return
    cursor.execute("INSERT INTO Members (name, contact_number) VALUES (%s, %s)", (name, contact))
    conn.commit()
    messagebox.showinfo("Success", "Member added!")

def issue_book():
    book_id = entry_issue_book.get()
    member_id = entry_issue_member.get()
    today = datetime.today().date()

    cursor.execute("SELECT available FROM Books WHERE book_id = %s", (book_id,))
    result = cursor.fetchone()
    if result and result[0]:
        cursor.execute("INSERT INTO IssueRecords (book_id, member_id, issue_date) VALUES (%s, %s, %s)",
                       (book_id, member_id, today))
        cursor.execute("UPDATE Books SET available = FALSE WHERE book_id = %s", (book_id,))
        conn.commit()
        messagebox.showinfo("Issued", "Book issued!")
    else:
        messagebox.showwarning("Unavailable", "Book is not available.")

def return_book():
    book_id = entry_return_book.get()
    today = datetime.today().date()
    cursor.execute("UPDATE IssueRecords SET return_date = %s WHERE book_id = %s AND return_date IS NULL", (today, book_id))
    cursor.execute("UPDATE Books SET available = TRUE WHERE book_id = %s", (book_id,))
    conn.commit()
    messagebox.showinfo("Returned", "Book returned!")

def check_available_books():
    cursor.execute("SELECT book_id, book_name, author FROM Books WHERE available = TRUE")
    books = cursor.fetchall()
    if not books:
        messagebox.showinfo("Available Books", "No books available.")
    else:
        display = "\n".join([f"{b[0]} - {b[1]} by {b[2]}" for b in books])
        messagebox.showinfo("Available Books", display)

def view_member_history():
    member_id = entry_history_member.get()
    cursor.execute("""
        SELECT Books.book_name, IssueRecords.issue_date, IssueRecords.return_date
        FROM IssueRecords
        JOIN Books ON IssueRecords.book_id = Books.book_id
        WHERE member_id = %s
    """, (member_id,))
    records = cursor.fetchall()
    if not records:
        messagebox.showinfo("History", "No history found.")
    else:
        display = "\n".join([f"{r[0]} | Issued: {r[1]} | Returned: {r[2]}" for r in records])
        messagebox.showinfo("History", display)

# UI Setup
root = tk.Tk()
root.title("📚 Library Management System")
root.configure(bg="#f0f8ff")

# Styling function
def label(text, row, col):
    return tk.Label(root, text=text, bg="#f0f8ff", fg="#333", font=("Arial", 10, "bold")).grid(row=row, column=col, sticky="w", pady=2)

# Add Book Section
label("Book Name", 0, 0)
entry_book_name = tk.Entry(root)
entry_book_name.grid(row=0, column=1)

label("Author", 1, 0)
entry_author = tk.Entry(root)
entry_author.grid(row=1, column=1)

tk.Button(root, text="Add Book", command=add_book, bg="#4caf50", fg="white").grid(row=2, column=1, pady=5)

# Add Member Section
label("Member Name", 3, 0)
entry_member = tk.Entry(root)
entry_member.grid(row=3, column=1)

label("Contact Number", 4, 0)
entry_contact = tk.Entry(root)
entry_contact.grid(row=4, column=1)

tk.Button(root, text="Add Member", command=add_member, bg="#2196f3", fg="white").grid(row=5, column=1, pady=5)

# Issue Book
label("Book ID to Issue", 6, 0)
entry_issue_book = tk.Entry(root)
entry_issue_book.grid(row=6, column=1)

label("Member ID", 7, 0)
entry_issue_member = tk.Entry(root)
entry_issue_member.grid(row=7, column=1)

tk.Button(root, text="Issue Book", command=issue_book, bg="#ff9800", fg="white").grid(row=8, column=1, pady=5)

# Return Book
label("Book ID to Return", 9, 0)
entry_return_book = tk.Entry(root)
entry_return_book.grid(row=9, column=1)

tk.Button(root, text="Return Book", command=return_book, bg="#9c27b0", fg="white").grid(row=10, column=1, pady=5)

# Check Available
tk.Button(root, text="Check Available Books", command=check_available_books, bg="#009688", fg="white").grid(row=11, column=1, pady=5)

# Member History
label("Member ID for History", 12, 0)
entry_history_member = tk.Entry(root)
entry_history_member.grid(row=12, column=1)

tk.Button(root, text="View Member History", command=view_member_history, bg="#795548", fg="white").grid(row=13, column=1, pady=5)

root.mainloop()
